# Свёрточные нейросети — классификация изображений (PyTorch)
### 1. Инициализация

In [ ]:
# Импорт необходимых библиотек и модулей
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Проверка версии PyTorch
torch.__version__

### 2. Загрузка и преобразование датасета
Здесь используется датасет [CIFAR‑10](https://www.cs.toronto.edu/~kriz/cifar.html) — популярный бенчмарк в компьютерном зрении. Он содержит 60 000 цветных изображений 32×32 в 10 классах (по 6 000 изображений на класс) и часто применяется для обучения и оценки моделей классификации изображений.

Классы: самолёт, автомобиль, птица, кошка, олень, собака, лягушка, лошадь, корабль и грузовик.

Перед загрузкой данных зададим аугментации и преобразования, чтобы снизить переобучение и улучшить обобщающую способность CNN.

Трансформации обучающих изображений:
- Случайные повороты;
- Случайные горизонтальные отражения;
- Случайные изменения яркости/контраста/насыщенности/оттенка (color jitter);
- Преобразование в тензор и нормализация значений.

Тестовые изображения не аугментируются: мы только приводим их к тензору и нормируем, чтобы честно оценивать качество.

Разделив конвейеры преобразований для обучения и теста, мы обучаемся на «обогащённых» данных, а оцениваемся на стабильных и неизменённых примерах.

In [ ]:
# Создание трансформера для обучающих данных как последовательности шагов
transform_train = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Создание трансформера для тестовых данных (без аугментации)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [ ]:
# Загрузка датасета из библиотеки PyTorch
data_train = torchvision.datasets.CIFAR10(root="../data/raw", train=True, download=True, transform=transform_train)
data_test = torchvision.datasets.CIFAR10(root="../data/raw", train=False, download=True, transform=transform_test)

# Инициализация загрузчиков данных
train_loader = torch.utils.data.DataLoader(data_train, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=32, shuffle=False)

### 3. Построение модели CNN

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        
        super(CNN, self).__init__()
        
        # Определяем свёрточные слои
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # Определяем слои пуллинга
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Определяем полносвязные слои
        self.fc1 = nn.Linear(64 * 8 * 8, 128)  # 8 * 8 * 6 — «сплющенное» измерение после пуллинга (из исходного комментария)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)  # 10 выходов под 10 классов

    def forward(self, x):
        
        # Применяем свёртки с ReLU и пуллингом
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        # «Сплющиваем» выход свёрточных слоёв
        x = x.view(-1, 64 * 8 * 8)
        
        # Полносвязные слои с ReLU
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        # Выходной слой (логиты классов)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

# Инициализируем модель
cnn = CNN()

# Функция потерь
criterion = nn.CrossEntropyLoss()

# Оптимизатор (Adam)
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)

### 4. Обучение модели CNN

In [ ]:
# Инициализация функции расчёта метрик
def compute_metrics(outputs, labels):
     # Преобразуем выходы модели в предсказанные метки
    _, preds = torch.max(outputs, 1)
    
    # Перенос меток и предсказаний на CPU и в numpy
    labels = labels.cpu().numpy()
    preds = preds.cpu().numpy()
    
    # Вычисляем accuracy, F1, precision и recall
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted', zero_division=1)
    recall = recall_score(labels, preds, average='weighted', zero_division=1)
    
    return accuracy, f1, precision, recall

In [ ]:
# Число эпох обучения
num_epochs = 25

# Проверяем доступность GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Переносим модель на доступное устройство
cnn.to(device)

# Переключаем модель в режим обучения
cnn.train()

# Списки для хранения метрик обучения и валидации
train_accuracies = []
val_accuracies = []

# Основной цикл обучения
for epoch in range(num_epochs):
    
    running_loss = 0.0
    running_accuracy = 0.0
    running_f1 = 0.0
    running_precision = 0.0
    running_recall = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        
         # Переносим батч на устройство
        inputs, labels = inputs.to(device), labels.to(device)     
        
        # Обнуляем градиенты параметров
        optimizer.zero_grad()
        
        # Прямой проход: считаем выход модели
        outputs = cnn(inputs)
        
        # Считаем функцию потерь
        loss = criterion(outputs, labels)
        
        # Обратное распространение: считаем градиенты
        loss.backward()
        
        # Обновляем параметры модели
        optimizer.step()
        
        # Накопление лосса
        running_loss += loss.item()
        
        # Вычисляем метрики
        accuracy, f1, precision, recall = compute_metrics(outputs, labels)
        running_accuracy += accuracy
        running_f1 += f1
        running_precision += precision
        running_recall += recall

    # Средние метрики за эпоху
    avg_loss = running_loss / len(train_loader)
    avg_accuracy = running_accuracy / len(train_loader)
    avg_f1 = running_f1 / len(train_loader)
    avg_precision = running_precision / len(train_loader)
    avg_recall = running_recall / len(train_loader)
    
#     Сохраняем точность обучения
    train_accuracies.append(avg_accuracy)
    
   # Переключаемся в режим валидации
    cnn.eval()
    
    # Инициализация метрик на валидации
    eval_loss = 0.0
    eval_accuracy = 0.0
    eval_f1 = 0.0
    eval_precision = 0.0
    eval_recall = 0.0
    
    # На валидации градиенты не считаем
    with torch.no_grad():
        for inputs, labels in test_loader:
            
            # Переносим батч на устройство
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Прямой проход: выход модели
            outputs = cnn(inputs)
            
            # Функция потерь
            loss = criterion(outputs, labels)
            
            # Накопление лосса на валидации
            eval_loss += loss.item()
            
            # Метрики
            accuracy, f1, precision, recall = compute_metrics(outputs, labels)
            eval_accuracy += accuracy
            eval_f1 += f1
            eval_precision += precision
            eval_recall += recall
    
    # Средние метрики на валидации
    avg_eval_loss = eval_loss / len(test_loader)
    avg_eval_accuracy = eval_accuracy / len(test_loader)
    avg_eval_f1 = eval_f1 / len(test_loader)
    avg_eval_precision = eval_precision / len(test_loader)
    avg_eval_recall = eval_recall / len(test_loader)
    
    # Сохраняем точность валидации
    val_accuracies.append(avg_eval_accuracy)
    
    # Печать усреднённых метрик
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'accuracy: {avg_accuracy:.4f} - f1_score: {avg_f1:.4f} - loss: {avg_loss:.4f} - '
          f'precision: {avg_precision:.4f} - recall: {avg_recall:.4f} - '
          f'val_accuracy: {avg_eval_accuracy:.4f} - val_f1_score: {avg_eval_f1:.4f} - val_loss: {avg_eval_loss:.4f} - '
          f'val_precision: {avg_eval_precision:.4f} - val_recall: {avg_eval_recall:.4f}')
        
    # Возвращаемся в режим обучения
    cnn.train()

### 5. Анализ качества

In [ ]:
plt.plot(train_accuracies)
plt.plot(val_accuracies)
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Результаты указывают на следующее:
- Точность на обучении стабильно растёт по мере увеличения числа эпох — модель действительно учится.
- Валидационная точность сначала улучшается, но затем колеблется — признак начинающегося переобучения.
- После пика (около 9-й эпохи) разрыв между обучением и валидацией увеличивается.
- Чтобы снизить переобучение, стоит рассмотреть регуляризацию (например, dropout, weight decay), раннюю остановку и/или усиление аугментаций.

**Итог.** Модель уверенно осваивает тренировочные данные, но со временем переобучается. Для лучшей обобщаемости полезно добавить регуляризацию, подобрать гиперпараметры и контролировать раннюю остановку.